In [ ]:
from jupyter_imports import *

In [ ]:
from riskpnl.post_trade import batch_summarize_exec_logs # use 'prod' for remote after cache_tmp

dirname = os.path.join(os.sep, 'tmp','tradeexecutor')#,'prod')
start = datetime(2022,6,20)#,tzinfo=timezone.utc)
log_file = os.path.join(os.sep, dirname,'all_by_symbol.csv')
batch_summarize_exec_logs(dirname,start=start,add_history_context=False,rebuild=True)
original_weights = pd.read_csv(log_file).sort_values(by='log_time')
original_weights['log_time'] = original_weights['log_time'].apply(parser.parse)

In [ ]:
weights = original_weights[original_weights['log_time']>start.replace(tzinfo=timezone.utc)]

weights['total_bps'] = weights['slippage_bps'] + weights['fee']
weights['fee_USD'] = weights['fee']*weights['filledUSD'].apply(np.abs)/10000 # in bps for by_symbol, in USD for by_clientorderid :(
weights['slippage_USD'] = weights['slippage_bps']*weights['filledUSD'].apply(np.abs)/10000
weights['total_USD'] = weights['total_bps']*weights['filledUSD'].apply(np.abs)/10000

### cumulative slippage and fees

In [ ]:
weights

In [ ]:
trackrecord = weights[weights['index']=='average'].pivot_table(index='log_time',values=['fee_USD','slippage_USD','total_USD','filledUSD'],aggfunc=np.mean).cumsum()

In [ ]:
trackrecord.iplot(secondary_y='filledUSD')

### slippage distribution

In [ ]:
(weights.loc[weights['index']=='average','total_bps']).hist(bins=50,figsize=(20,10))

In [ ]:
original_coin = pd.read_csv(os.path.join(os.sep, dirname,'all_by_coin.csv')).sort_values(by='log_time')

In [ ]:
original_coin

In [ ]:
trackrecord = original_coin[original_coin['index']=='average'].pivot_table(index='log_time',values=['perleg_fee_bps','perleg_filled_USD','premium_vs_inception_bps'],aggfunc=np.mean)
trackrecord.iplot(secondary_y='perleg_filled_USD')

# latency

In [ ]:
original_clientOrderId = pd.read_csv(os.path.join(os.sep, dirname,'all_by_clientOrderId.csv')).sort_values(by='log_time')
original_data = pd.read_csv(os.path.join(os.sep, dirname,'all_data.csv')).sort_values(by='log_time')

In [ ]:
df = original_clientOrderId[['tick_to_order_local']]
df['local_lag'] = original_data['timestamp']-original_data['remote_timestamp']
df.hist(bins=50,figsize=(20,10))
df.describe()